<a href="https://colab.research.google.com/github/samaaMuhamad/spark-task/blob/main/SparkTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.3.1/spark-2.3.1-bin-hadoop2.7.tgz
!tar xf spark-2.3.1-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.1-bin-hadoop2.7"

!ls

import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate() 
spark


Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [787 kB]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelea

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Learning_Spark") \
    .getOrCreate()



#query 1 

sc = spark.sparkContext

listKamla=[]  #de 3mlaha 3shan a7ot feha el integrs kolha w b3den a7wel el list be paralelize le rdd
#w fel akher a3mel union lel rdd da m3 el rdds bto3 query 1 w 5
lines = sc.textFile("pagecounts-20160101-000000_parsed.out").persist()   #3shan hafdal ast3melo ta7t fa 3mlto persist


print(lines.take(10)) #action
res=lines.take(10)
res=["Query1"]+res
resultQ1=sc.parallelize(res) #action

#parallize bt7wel list le rdd 3shan a2dr a3melo save as text file


##############################################################

#query 2

def parseLine(line):
    fields = line.split(' ')
    pageSize = int(fields[3])
    return (pageSize)

pagesizes = lines.map(parseLine).persist()  #de transformation

print("Query 2")
listKamla.append("Query2")
#orderedSmallToLarge= pagesizes.takeOrdered(1) 
minimum=pagesizes.reduce(lambda a,b:min(a,b)) #action
print("Minimum of page sizes: ",minimum)
listKamla.append(minimum)


#orderedLargeToSmall= pagesizes.takeOrdered(1, key = lambda x: -x[1])
maximum=pagesizes.reduce(lambda a,b:max(a,b)) #action
print("Maximum of page sizes: ",maximum)
listKamla.append(maximum)

summation=pagesizes.reduce(lambda a,b:a +b)
countData=pagesizes.count()
print("Average of page sizes: ", summation/countData)        
listKamla.append(summation/countData)
################################################################

#query 3

def parseLineTwo(line):
  fields = line.split(' ')
  pageTitle = fields[1]
  projectCode = fields[0]
  return (projectCode,pageTitle)

def startwith(line):
  if(line[1].startswith("The")):
    return line
def notEnglishProject(line):
  if(line[0] != 'en'):
    return line


print("Query 3")
listKamla.append("Query3")

pageTitlesAndCodes = lines.map(parseLineTwo) #transformation

pagesTitlesStartWithThe=pageTitlesAndCodes.filter(startwith) #transformation

print("Page titles that start with The : ",pagesTitlesStartWithThe.count()) #action
listKamla.append(pagesTitlesStartWithThe.count())


notEnglish= pagesTitlesStartWithThe.filter(notEnglishProject)
print("Page titles that start with The and not english project : ",notEnglish.count())    
listKamla.append(notEnglish.count())
#####################################################################


#query 4

listKamla.append("Query 4")
def parseLineQuery1(line):
  fields = line.split(' ')
  pageTitle = fields[1] 
  return pageTitle

def parseTerms(title):
  allFields= title.split('_')
  return allFields



def enterr(line):
  for t in line:
    termsNew=''.join(filter(str.isalnum, t.lower()))
  return termsNew



pageTitlesssAll = lines.map(parseLineQuery1)
pageTerms=pageTitlesssAll.map(parseTerms)
termsNN=pageTerms.map(enterr)

print("Unique terms in page titles are: " ,termsNN.take(10))
uniq=termsNN.distinct()




print("Query 4")


print("Number of uniue page terms " , uniq.count())

listKamla.append(uniq.count())
######################################################

#query 5
def parseLineQuery(line):
  fields = line.split(' ')
  pageTitle = ''.join(filter(str.isalnum, fields[1].lower()))  #.lower btkhlleeha small, el kalam ely ablaha da
  return (pageTitle)  #by3mel check str.isalnum 
#isalnum() function, which checks whether a given string contains alphanumeric characters or not by checking each character. 
#The join() function combines all the characters to return a string.


pageTitles = lines.map(parseLineQuery).persist()    #3shan hast3melo ta7t fa 3mlto persist



print("Query 5")

pageTitlesCounts = pageTitles.map(lambda word: (word, 1)).reduceByKey(lambda a,b:a +b)
#pageTitlesCounts.saveAsTextFile("titlecountFinalKhaless4")

resultt=pageTitlesCounts.takeOrdered(2, key = lambda x: -x[1]) #take ordered de action Returns the first n elements of the RDD 
resultt=["Query5"]+resultt
print(resultt)
resultQ5=sc.parallelize(resultt)  #parallize bt7wel list le rdd 3shan a2dr a3melo save as text file

#using either their natural order or a custom comparator
#lw 3yzen a3la wa7da bas hanakhly 5 be 1
#ana hena b3mel zy query 4 w basheel ay 7aga msh alphanumericc w bakhlehom lowercase
listKamla2=sc.parallelize(listKamla)
resulttFinal=resultQ1.union(listKamla2)
resulttFinal2=resulttFinal.union(resultQ5)

resulttFinal2.saveAsTextFile('LastFile')

###########################################################################


['aa 271_a.C 1 4675', 'aa Category:User_th 1 4770', 'aa Chiron_Elias_Krase 1 4694', 'aa Dassault_rafaele 2 9372', 'aa E.Desv 1 4662', 'aa File:Wiktionary-logo-en.png 1 10752', 'aa Indonesian_Wikipedia 1 4679', 'aa Main_Page 5 266946', 'aa Requests_for_new_languages/Wikipedia_Banyumasan 1 4733', 'aa Special:Contributions/203.144.160.245 1 5812']
Query 2
Minimum of page sizes:  0
Maximum of page sizes:  141180155987
Average of page sizes:  132239.56957446598
Query 3
Page titles that start with The :  45020
Page titles that start with The and not english project :  10292
Unique terms in page titles are:  ['ac', 'th', 'krase', 'rafaele', 'edesv', 'filewiktionarylogoenpng', 'wikipedia', 'page', 'banyumasan', 'specialcontributions203144160245']
Query 4
Number of uniue page terms  1218892
Query 5
['Query5', ('', 207), ('water', 127)]
